In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
loss_fn = nn.CrossEntropyLoss()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304471  [    0/60000]
loss: 2.301991  [ 6400/60000]
loss: 2.286296  [12800/60000]
loss: 2.277086  [19200/60000]
loss: 2.287848  [25600/60000]
loss: 2.263774  [32000/60000]
loss: 2.274053  [38400/60000]
loss: 2.257900  [44800/60000]
loss: 2.252005  [51200/60000]
loss: 2.237609  [57600/60000]
Test Error: 
 Accuracy: 30.5%, Avg loss: 2.233625 

Epoch 2
-------------------------------
loss: 2.249748  [    0/60000]
loss: 2.258980  [ 6400/60000]
loss: 2.205883  [12800/60000]
loss: 2.196775  [19200/60000]
loss: 2.237684  [25600/60000]
loss: 2.183932  [32000/60000]
loss: 2.213146  [38400/60000]
loss: 2.177525  [44800/60000]
loss: 2.166194  [51200/60000]
loss: 2.147497  [57600/60000]
Test Error: 
 Accuracy: 31.8%, Avg loss: 2.134135 

Epoch 3
-------------------------------
loss: 2.169045  [    0/60000]
loss: 2.186168  [ 6400/60000]
loss: 2.075048  [12800/60000]
loss: 2.063086  [19200/60000]
loss: 2.156035  [25600/60000]
loss: 2.063077  [32000/600